# Dimensionality Reduction & Generative Models in PyTorch

This notebook walks through three fundamental techniques for learning compact representations of data:

| Method | Type | Latent Space | Generative? |
|--------|------|-------------|-------------|
| **PCA** | Linear projection | Orthogonal principal components | No |
| **Autoencoder** | Nonlinear compression | Unconstrained bottleneck | Weak |
| **VAE** | Probabilistic model | Regularised Gaussian latent | Yes |

All three share a common thread: find a **low-dimensional representation** $\mathbf{z}$ of high-dimensional data $\mathbf{x}$, such that $\mathbf{x}$ can be (approximately) recovered from $\mathbf{z}$.

We use **MNIST** throughout so results are directly comparable.

In [ ]:
# Colab setup — installs are no-ops if packages already exist
import importlib, subprocess, sys

for pkg in ["torch", "torchvision", "matplotlib", "numpy"]:
    if importlib.util.find_spec(pkg) is None:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

# Enable GPU: Runtime → Change runtime type → T4 GPU
import torch
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Running on CPU — consider enabling a GPU runtime for faster training.")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.manual_seed(42)
np.random.seed(42)

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

IMG_DIM = 28 * 28
LATENT_DIM = 2  # 2-D latent for easy visualisation

print(f"Train: {len(train_dataset)} | Test: {len(test_dataset)} | Image dim: {IMG_DIM}")

In [ ]:
def show_images(originals, reconstructions, n=8, title=""):
    """Side-by-side comparison of original vs reconstructed images."""
    fig, axes = plt.subplots(2, n, figsize=(n * 1.5, 3))
    for i in range(n):
        axes[0, i].imshow(originals[i].reshape(28, 28), cmap="gray")
        axes[0, i].axis("off")
        axes[1, i].imshow(reconstructions[i].reshape(28, 28), cmap="gray")
        axes[1, i].axis("off")
    axes[0, 0].set_title("Original", fontsize=10)
    axes[1, 0].set_title("Reconstructed", fontsize=10)
    fig.suptitle(title, fontsize=13, fontweight="bold")
    plt.tight_layout()
    plt.show()


def plot_latent_space(z, labels, title=""):
    """Scatter plot of 2-D latent codes coloured by digit class."""
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(z[:, 0], z[:, 1], c=labels, cmap="tab10", s=2, alpha=0.6)
    plt.colorbar(scatter, ticks=range(10))
    plt.title(title, fontsize=14, fontweight="bold")
    plt.xlabel("$z_1$")
    plt.ylabel("$z_2$")
    plt.tight_layout()
    plt.show()

---
## 1. PCA — Principal Component Analysis in PyTorch

### Theory

PCA finds an orthonormal basis $\{\mathbf{v}_1, \dots, \mathbf{v}_k\}$ that maximises variance of the projected data.

Given centred data matrix $\mathbf{X} \in \mathbb{R}^{n \times d}$:

$$\mathbf{C} = \frac{1}{n-1}\mathbf{X}^\top \mathbf{X}$$

The top-$k$ eigenvectors of $\mathbf{C}$ (or equivalently, the right singular vectors from SVD of $\mathbf{X}$) form the projection:

$$\mathbf{z} = \mathbf{x} \, \mathbf{V}_k, \qquad \hat{\mathbf{x}} = \mathbf{z} \, \mathbf{V}_k^\top$$

### Key properties
- **Linear** — the encoder and decoder are both matrix multiplications
- **Globally optimal** under MSE for linear projections (Eckart–Young theorem)
- **No training loop** — solved via eigendecomposition / SVD in one shot

In [ ]:
class PCA:
    """PCA via SVD, implemented entirely in PyTorch."""

    def __init__(self, n_components: int):
        self.k = n_components
        self.mean = None
        self.components = None  # (d, k)
        self.singular_values = None
        self.explained_variance_ratio = None

    @torch.no_grad()
    def fit(self, X: torch.Tensor):
        """Fit PCA on data X of shape (n, d)."""
        self.mean = X.mean(dim=0)
        X_centered = X - self.mean

        U, S, Vt = torch.linalg.svd(X_centered, full_matrices=False)

        self.components = Vt[:self.k].T                # (d, k)
        self.singular_values = S[:self.k]

        total_var = (S ** 2).sum()
        self.explained_variance_ratio = (S[:self.k] ** 2) / total_var
        return self

    @torch.no_grad()
    def transform(self, X: torch.Tensor) -> torch.Tensor:
        return (X - self.mean) @ self.components

    @torch.no_grad()
    def inverse_transform(self, Z: torch.Tensor) -> torch.Tensor:
        return Z @ self.components.T + self.mean

In [ ]:
X_train = train_dataset.data.float().reshape(-1, IMG_DIM) / 255.0
X_test = test_dataset.data.float().reshape(-1, IMG_DIM) / 255.0
y_test = test_dataset.targets.numpy()

pca = PCA(n_components=LATENT_DIM).fit(X_train)

cumulative = pca.explained_variance_ratio.cumsum(0)
print(f"Explained variance (top {LATENT_DIM} components): {cumulative[-1]:.4f}")

In [ ]:
# Variance explained vs number of components
pca_full = PCA(n_components=50).fit(X_train)
cumvar = pca_full.explained_variance_ratio.cumsum(0).numpy()

plt.figure(figsize=(7, 4))
plt.plot(range(1, 51), cumvar, "o-", markersize=4)
plt.xlabel("Number of components")
plt.ylabel("Cumulative explained variance")
plt.title("PCA — Explained Variance", fontweight="bold")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
z_pca = pca.transform(X_test)
x_recon_pca = pca.inverse_transform(z_pca)

pca_mse = F.mse_loss(x_recon_pca, X_test).item()
print(f"PCA reconstruction MSE: {pca_mse:.4f}")

show_images(X_test.numpy(), x_recon_pca.numpy(), title="PCA Reconstruction (2-D)")
plot_latent_space(z_pca.numpy(), y_test, title="PCA Latent Space (2-D)")

---
## 2. Autoencoder

### Theory

An autoencoder learns *nonlinear* encoder $f_\phi$ and decoder $g_\theta$ to minimise reconstruction error:

$$\mathcal{L}_{\text{AE}} = \|\mathbf{x} - g_\theta(f_\phi(\mathbf{x}))\|^2$$

With a bottleneck layer of dimension $k \ll d$, the network is forced to learn a compressed representation.

### PCA as a special case
A **linear** autoencoder (no activations) with MSE loss learns the same subspace as PCA. The nonlinear activations let us capture more complex structure.

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim: int = IMG_DIM, latent_dim: int = LATENT_DIM):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, input_dim),
            nn.Sigmoid(),
        )

    def encode(self, x):
        return self.encoder(x)

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        z = self.encode(x)
        return self.decode(z), z

In [ ]:
def train_autoencoder(model, train_loader, epochs=20, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()
    history = []

    for epoch in range(1, epochs + 1):
        total_loss = 0.0
        for batch, _ in train_loader:
            x = batch.view(-1, IMG_DIM).to(device)
            x_hat, _ = model(x)
            loss = F.mse_loss(x_hat, x)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * x.size(0)

        avg = total_loss / len(train_loader.dataset)
        history.append(avg)
        if epoch % 5 == 0 or epoch == 1:
            print(f"  Epoch {epoch:3d} | MSE: {avg:.4f}")

    return history

In [ ]:
ae = Autoencoder().to(device)
print(f"Autoencoder parameters: {sum(p.numel() for p in ae.parameters()):,}")
print()
ae_history = train_autoencoder(ae, train_loader, epochs=20)

In [ ]:
plt.figure(figsize=(7, 4))
plt.plot(ae_history, "o-", markersize=4)
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Autoencoder Training", fontweight="bold")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
ae.eval()
with torch.no_grad():
    x_test_t = X_test.to(device)
    x_recon_ae, z_ae = ae(x_test_t)
    x_recon_ae = x_recon_ae.cpu().numpy()
    z_ae = z_ae.cpu().numpy()

ae_mse = np.mean((X_test.numpy() - x_recon_ae) ** 2)
print(f"Autoencoder reconstruction MSE: {ae_mse:.4f}")

show_images(X_test.numpy(), x_recon_ae, title="Autoencoder Reconstruction (2-D)")
plot_latent_space(z_ae, y_test, title="Autoencoder Latent Space (2-D)")

---
## 3. Variational Autoencoder (VAE)

### 3.1 The goal: learn a generative model

We want to learn a model that can **generate** new data like our training images.  
The natural objective is **maximum likelihood** — find parameters $\theta$ that maximise:

$$p_\theta(\mathbf{x}) = \prod_{i=1}^{N} p_\theta(\mathbf{x}^{(i)})$$

or equivalently maximise $\log p_\theta(\mathbf{x})$ for each data point.

---

### 3.2 Introducing latent variables — the joint probability

We assume each image $\mathbf{x}$ was generated from an unobserved latent code $\mathbf{z}$:

$$\boxed{p_\theta(\mathbf{x}, \mathbf{z}) = p_\theta(\mathbf{x}|\mathbf{z}) \; p(\mathbf{z})}$$

- **Prior:** $p(\mathbf{z}) = \mathcal{N}(\mathbf{0}, \mathbf{I})$ — simple Gaussian, easy to sample from
- **Likelihood / decoder:** $p_\theta(\mathbf{x}|\mathbf{z})$ — a neural network that maps $\mathbf{z}$ to an image

The **marginal likelihood** (the quantity we actually want to maximise) requires integrating out $\mathbf{z}$:

$$p_\theta(\mathbf{x}) = \int p_\theta(\mathbf{x}|\mathbf{z}) \; p(\mathbf{z}) \; d\mathbf{z}$$

---

### 3.3 The problem: why direct optimisation fails

**Problem 1 — Intractable integral.**  
The integral $\int p_\theta(\mathbf{x}|\mathbf{z}) \, p(\mathbf{z}) \, d\mathbf{z}$ sums over *every possible* $\mathbf{z}$. When the decoder is a neural network this has no closed form. We could try Monte Carlo:

$$p_\theta(\mathbf{x}) \approx \frac{1}{K}\sum_{k=1}^{K} p_\theta(\mathbf{x}|\mathbf{z}^{(k)}), \quad \mathbf{z}^{(k)} \sim p(\mathbf{z})$$

but most random $\mathbf{z}$ samples will produce images nothing like $\mathbf{x}$, so $p_\theta(\mathbf{x}|\mathbf{z}^{(k)}) \approx 0$ for almost all samples — the estimate has **astronomically high variance**.

**Problem 2 — Intractable posterior.**  
If we could compute the true posterior $p_\theta(\mathbf{z}|\mathbf{x}) = \frac{p_\theta(\mathbf{x}|\mathbf{z})\,p(\mathbf{z})}{p_\theta(\mathbf{x})}$ we could sample the "right" $\mathbf{z}$ values. But computing it requires the very marginal $p_\theta(\mathbf{x})$ we can't evaluate.

> **Bottom line:** We can write down $p_\theta(\mathbf{x}, \mathbf{z})$ easily, but we can neither evaluate nor maximise $p_\theta(\mathbf{x})$ directly.

Let's see this concretely below before introducing the fix.

### Demonstration: why naive Monte Carlo on the joint fails

Below we build a simple decoder, pick a real image $\mathbf{x}$, and estimate $p_\theta(\mathbf{x})$ by sampling $\mathbf{z}$ from the prior.  Watch how the estimate is essentially **zero** — almost no random $\mathbf{z}$ produces anything close to the target image.

In [ ]:
class SimpleDecoder(nn.Module):
    """A small decoder to demonstrate the intractability of the marginal."""
    def __init__(self, latent_dim=LATENT_DIM, output_dim=IMG_DIM):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, 128), nn.ReLU(),
            nn.Linear(128, 256), nn.ReLU(),
            nn.Linear(256, output_dim), nn.Sigmoid(),
        )
    def forward(self, z):
        return self.net(z)

demo_decoder = SimpleDecoder().to(device)

# Pick one real test image
x_target = X_test[0:1].to(device)  # shape (1, 784)

# --- Naive Monte Carlo: sample z ~ N(0,I), compute p(x|z) ---
# Model p(x|z) as independent Bernoulli per pixel:
#   log p(x|z) = sum_d [ x_d log mu_d + (1-x_d) log(1-mu_d) ]

K_values = [10, 100, 1_000, 10_000]
print("Naive Monte Carlo estimate of log p(x)  [sampling z from prior]\n")

for K in K_values:
    with torch.no_grad():
        z_samples = torch.randn(K, LATENT_DIM).to(device)
        mu = demo_decoder(z_samples)                             # (K, 784)
        mu = mu.clamp(1e-6, 1 - 1e-6)
        # log p(x|z) for each sample — sum over pixels
        log_px_given_z = (x_target * mu.log() + (1 - x_target) * (1 - mu).log()).sum(dim=1)  # (K,)
        # log p(x) ≈ log (1/K Σ exp(log p(x|z_k)))  via log-sum-exp
        log_px_estimate = torch.logsumexp(log_px_given_z, dim=0) - np.log(K)
    print(f"  K = {K:>6,}  →  log p(x) ≈ {log_px_estimate.item():>10.1f}   "
          f"(best single sample: {log_px_given_z.max().item():.1f})")

print("\n→ Even with 10 000 samples the estimate is dominated by noise.")
print("  The true log p(x) should be around -100 to -200 for MNIST,")
print("  but almost every z from the prior produces garbage → p(x|z) ≈ 0.")

In [ ]:
# Visualise: what does the prior produce vs the target?
fig, axes = plt.subplots(2, 9, figsize=(14, 3.2))

axes[0, 0].imshow(x_target.cpu().numpy().reshape(28, 28), cmap="gray")
axes[0, 0].set_title("Target x", fontsize=9, fontweight="bold")
axes[0, 0].axis("off")

with torch.no_grad():
    z_rand = torch.randn(8, LATENT_DIM).to(device)
    decoded_rand = demo_decoder(z_rand).cpu().numpy()

for i in range(8):
    axes[0, i + 1].imshow(decoded_rand[i].reshape(28, 28), cmap="gray")
    axes[0, i + 1].set_title(f"z sample {i+1}", fontsize=8)
    axes[0, i + 1].axis("off")

# Bottom row: show the per-pixel log-likelihood — almost all near -∞
with torch.no_grad():
    mu_rand = demo_decoder(z_rand).clamp(1e-6, 1 - 1e-6)
    ll = (x_target.cpu() * mu_rand.cpu().log() + (1 - x_target.cpu()) * (1 - mu_rand.cpu()).log()).sum(dim=1)

axes[1, 0].axis("off")
for i in range(8):
    axes[1, i + 1].text(0.5, 0.5, f"log p(x|z)\n{ll[i].item():.0f}",
                         ha="center", va="center", fontsize=9, transform=axes[1, i + 1].transAxes)
    axes[1, i + 1].axis("off")

fig.suptitle("Random prior samples produce images nothing like x  →  p(x|z) ≈ 0",
             fontsize=12, fontweight="bold")
plt.tight_layout()
plt.show()

print("This is WHY we need an encoder q(z|x) that proposes 'good' z values for each x.")
print("That insight leads us to the ELBO (section 3.4 above).")

### 3.4 The solution: ELBO (Evidence Lower Bound)

**Key idea:** introduce an *approximate* posterior $q_\phi(\mathbf{z}|\mathbf{x})$ (an encoder network) that, given $\mathbf{x}$, proposes **useful** $\mathbf{z}$ values. Then derive a **lower bound** on $\log p_\theta(\mathbf{x})$ that *is* tractable.

**Derivation:**

Start from the log marginal likelihood and multiply/divide by $q_\phi$:

$$\log p_\theta(\mathbf{x}) = \log \int p_\theta(\mathbf{x}, \mathbf{z}) \, d\mathbf{z} = \log \int \frac{p_\theta(\mathbf{x}, \mathbf{z})}{q_\phi(\mathbf{z}|\mathbf{x})} \, q_\phi(\mathbf{z}|\mathbf{x}) \, d\mathbf{z}$$

Apply **Jensen's inequality** ($\log$ is concave, so $\log\mathbb{E}[X] \geq \mathbb{E}[\log X]$):

$$\log p_\theta(\mathbf{x}) \geq \int q_\phi(\mathbf{z}|\mathbf{x}) \log \frac{p_\theta(\mathbf{x}, \mathbf{z})}{q_\phi(\mathbf{z}|\mathbf{x})} \, d\mathbf{z} \;=\; \mathcal{L}(\theta, \phi; \mathbf{x})$$

Expand the joint $p_\theta(\mathbf{x}, \mathbf{z}) = p_\theta(\mathbf{x}|\mathbf{z})\,p(\mathbf{z})$ and split the log:

$$\boxed{\mathcal{L} = \underbrace{\mathbb{E}_{q_\phi(\mathbf{z}|\mathbf{x})}[\log p_\theta(\mathbf{x}|\mathbf{z})]}_{\text{Reconstruction term}} \;-\; \underbrace{D_{\text{KL}}\big(q_\phi(\mathbf{z}|\mathbf{x}) \,\|\, p(\mathbf{z})\big)}_{\text{KL regularisation}}}$$

**Why this fixes the problem:**

| Naive Monte Carlo (Section 3.3) | ELBO |
|---|---|
| Samples $\mathbf{z}$ blindly from prior $p(\mathbf{z})$ | Samples $\mathbf{z}$ from encoder $q_\phi(\mathbf{z}|\mathbf{x})$ — tailored to each $\mathbf{x}$ |
| Almost all $\mathbf{z}$ produce garbage for the given $\mathbf{x}$ | Encoder learns to propose $\mathbf{z}$ that reconstruct $\mathbf{x}$ well |
| Estimate has near-infinite variance | Low-variance gradient estimates via reparameterisation |

**Interpretation of the two terms:**
- **Reconstruction:** keep the decoder accurate — the encoder should propose $\mathbf{z}$'s that let the decoder recover $\mathbf{x}$
- **KL:** keep the encoder close to the prior $\mathcal{N}(\mathbf{0}, \mathbf{I})$ — this is what gives the latent space its smooth, sampleable structure (prevents the encoder from collapsing each $\mathbf{x}$ to a single point far from the prior)

The gap between $\log p_\theta(\mathbf{x})$ and the ELBO is exactly $D_{\text{KL}}(q_\phi \| p_\theta(\mathbf{z}|\mathbf{x})) \geq 0$, so tightening the ELBO simultaneously improves both the generative model and the approximate posterior.

---

### 3.5 Reparameterisation trick

We parameterise $q_\phi(\mathbf{z}|\mathbf{x}) = \mathcal{N}(\boldsymbol{\mu}_\phi(\mathbf{x}),\; \text{diag}(\boldsymbol{\sigma}_\phi^2(\mathbf{x})))$.

To backpropagate through the stochastic sampling, we reparameterise:

$$\mathbf{z} = \boldsymbol{\mu} + \boldsymbol{\sigma} \odot \boldsymbol{\epsilon}, \quad \boldsymbol{\epsilon} \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$$

Now the randomness is in $\boldsymbol{\epsilon}$ (independent of $\phi$), and $\mathbf{z}$ is a deterministic, differentiable function of $\phi$.

### 3.6 KL divergence (closed form)

For diagonal Gaussian $q$ vs standard Gaussian prior:

$$D_{\text{KL}} = -\frac{1}{2}\sum_{j=1}^{k}\left(1 + \log \sigma_j^2 - \mu_j^2 - \sigma_j^2\right)$$

### 3.7 Putting it together — VAE implementation

Now that we've seen:
1. **Joint probability** $p_\theta(\mathbf{x}, \mathbf{z}) = p_\theta(\mathbf{x}|\mathbf{z})\,p(\mathbf{z})$ — easy to write down
2. **Marginal** $p_\theta(\mathbf{x}) = \int p_\theta(\mathbf{x}|\mathbf{z})\,p(\mathbf{z})\,d\mathbf{z}$ — intractable to compute
3. **Naive Monte Carlo** — almost all prior samples give $p(\mathbf{x}|\mathbf{z}) \approx 0$
4. **ELBO** — a tractable lower bound using an encoder $q_\phi(\mathbf{z}|\mathbf{x})$

We implement the full VAE below. The encoder outputs $\boldsymbol{\mu}$ and $\log\boldsymbol{\sigma}^2$, we sample via the reparameterisation trick, decode, and optimise the ELBO.

In [ ]:
class VAE(nn.Module):
    def __init__(self, input_dim: int = IMG_DIM, latent_dim: int = LATENT_DIM):
        super().__init__()
        self.latent_dim = latent_dim

        # Shared encoder backbone
        self.encoder_backbone = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
        )
        # Two heads: mean and log-variance
        self.fc_mu = nn.Linear(128, latent_dim)
        self.fc_logvar = nn.Linear(128, latent_dim)

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, input_dim),
            nn.Sigmoid(),
        )

    def encode(self, x):
        h = self.encoder_backbone(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterise(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + std * eps

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterise(mu, logvar)
        return self.decode(z), mu, logvar, z

In [ ]:
def vae_loss(x_hat, x, mu, logvar, beta=1.0):
    """ELBO = reconstruction (BCE) + beta * KL divergence."""
    recon = F.binary_cross_entropy(x_hat, x, reduction="sum")
    kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon + beta * kl, recon, kl


def train_vae(model, train_loader, epochs=20, lr=1e-3, beta=1.0):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()
    history = {"total": [], "recon": [], "kl": []}

    for epoch in range(1, epochs + 1):
        t_total, t_recon, t_kl = 0.0, 0.0, 0.0
        for batch, _ in train_loader:
            x = batch.view(-1, IMG_DIM).to(device)
            x_hat, mu, logvar, _ = model(x)
            loss, recon, kl = vae_loss(x_hat, x, mu, logvar, beta)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            t_total += loss.item()
            t_recon += recon.item()
            t_kl += kl.item()

        n = len(train_loader.dataset)
        history["total"].append(t_total / n)
        history["recon"].append(t_recon / n)
        history["kl"].append(t_kl / n)

        if epoch % 5 == 0 or epoch == 1:
            print(f"  Epoch {epoch:3d} | ELBO: {t_total/n:.2f}  Recon: {t_recon/n:.2f}  KL: {t_kl/n:.2f}")

    return history

In [ ]:
vae = VAE().to(device)
print(f"VAE parameters: {sum(p.numel() for p in vae.parameters()):,}")
print()
vae_history = train_vae(vae, train_loader, epochs=30, beta=1.0)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for ax, key, color in zip(axes, ["total", "recon", "kl"], ["#2196F3", "#4CAF50", "#FF9800"]):
    ax.plot(vae_history[key], "o-", markersize=3, color=color)
    ax.set_xlabel("Epoch")
    ax.set_title(key.upper(), fontweight="bold")
    ax.grid(True, alpha=0.3)
fig.suptitle("VAE Training Curves", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

In [ ]:
vae.eval()
with torch.no_grad():
    x_test_t = X_test.to(device)
    x_recon_vae, mu_vae, _, z_vae = vae(x_test_t)
    x_recon_vae = x_recon_vae.cpu().numpy()
    z_vae_np = mu_vae.cpu().numpy()  # use mean for deterministic plot

vae_mse = np.mean((X_test.numpy() - x_recon_vae) ** 2)
print(f"VAE reconstruction MSE: {vae_mse:.4f}")

show_images(X_test.numpy(), x_recon_vae, title="VAE Reconstruction (2-D)")
plot_latent_space(z_vae_np, y_test, title="VAE Latent Space (2-D)")

### Generating new digits

Because the VAE latent space is regularised to match $\mathcal{N}(\mathbf{0}, \mathbf{I})$, we can **sample** from it to generate new images that never existed in the training set.

In [ ]:
vae.eval()
with torch.no_grad():
    # Sample from standard Gaussian
    z_samples = torch.randn(16, LATENT_DIM).to(device)
    generated = vae.decode(z_samples).cpu().numpy()

fig, axes = plt.subplots(2, 8, figsize=(12, 3))
for i, ax in enumerate(axes.flat):
    ax.imshow(generated[i].reshape(28, 28), cmap="gray")
    ax.axis("off")
fig.suptitle("VAE — Randomly Generated Digits", fontsize=13, fontweight="bold")
plt.tight_layout()
plt.show()

In [ ]:
# 2-D manifold: sweep z1 and z2 across the prior to see smooth transitions
n = 15
grid = np.linspace(-3, 3, n)
canvas = np.zeros((28 * n, 28 * n))

vae.eval()
with torch.no_grad():
    for i, yi in enumerate(grid):
        for j, xi in enumerate(grid):
            z = torch.tensor([[xi, yi]], dtype=torch.float32).to(device)
            img = vae.decode(z).cpu().numpy().reshape(28, 28)
            canvas[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = img

plt.figure(figsize=(10, 10))
plt.imshow(canvas, cmap="gray")
plt.title("VAE — Latent Space Manifold", fontsize=14, fontweight="bold")
plt.xlabel("$z_1$")
plt.ylabel("$z_2$")
plt.xticks(np.linspace(0, 28 * n, 7), [f"{v:.1f}" for v in np.linspace(-3, 3, 7)])
plt.yticks(np.linspace(0, 28 * n, 7), [f"{v:.1f}" for v in np.linspace(-3, 3, 7)])
plt.tight_layout()
plt.show()

---
## 4. Side-by-Side Comparison

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

for ax, z, title in zip(
    axes,
    [z_pca.numpy(), z_ae, z_vae_np],
    [f"PCA  (MSE={pca_mse:.4f})",
     f"Autoencoder  (MSE={ae_mse:.4f})",
     f"VAE  (MSE={vae_mse:.4f})"],
):
    scatter = ax.scatter(z[:, 0], z[:, 1], c=y_test, cmap="tab10", s=1, alpha=0.5)
    ax.set_title(title, fontsize=12, fontweight="bold")
    ax.set_xlabel("$z_1$")
    ax.set_ylabel("$z_2$")

plt.colorbar(scatter, ax=axes[-1], ticks=range(10), label="Digit")
fig.suptitle("Latent Space Comparison (2-D)", fontsize=15, fontweight="bold")
plt.tight_layout()
plt.show()

In [ ]:
# Reconstruction comparison on the same images
idx = [0, 3, 5, 7, 11, 18, 42, 61]
x_orig = X_test.numpy()[idx]

fig, axes = plt.subplots(4, len(idx), figsize=(len(idx) * 1.5, 6))
row_labels = ["Original", "PCA", "Autoencoder", "VAE"]
recons = [x_orig, x_recon_pca.numpy()[idx], x_recon_ae[idx], x_recon_vae[idx]]

for row, (imgs, label) in enumerate(zip(recons, row_labels)):
    for col in range(len(idx)):
        axes[row, col].imshow(imgs[col].reshape(28, 28), cmap="gray")
        axes[row, col].axis("off")
    axes[row, 0].set_ylabel(label, fontsize=11, fontweight="bold", rotation=0, labelpad=70, va="center")

fig.suptitle("Reconstruction Comparison", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

---
## 5. Summary

| | PCA | Autoencoder | VAE |
|---|---|---|---|
| **Mapping** | Linear | Nonlinear | Nonlinear + probabilistic |
| **Training** | Closed-form (SVD) | Gradient descent | Gradient descent |
| **Loss** | Reconstruction (MSE) | Reconstruction (MSE) | ELBO = Recon + KL |
| **Latent structure** | Orthogonal axes | Unconstrained | Regularised Gaussian |
| **Generation** | Not meaningful | Poor (holes in latent space) | Smooth sampling from $\mathcal{N}(0, I)$ |
| **Interpolation** | Linear only | May be discontinuous | Smooth and meaningful |

### Key takeaways

1. **PCA** gives the best *linear* compression. It's fast, parameter-free, and a great baseline.
2. **Autoencoders** can learn richer representations via nonlinear mappings, but their latent space has no structure — you can't sample from it to generate realistic data.
3. **VAEs** regularise the latent space to be a smooth Gaussian, enabling meaningful generation and interpolation at the cost of slightly blurrier reconstructions.

### Connection to LoRA

PCA's core insight — that high-dimensional data often lies near a *low-rank* subspace — directly motivates **LoRA**. 
LoRA approximates weight updates as $\Delta W = BA$ where $B \in \mathbb{R}^{d \times r}$ and $A \in \mathbb{R}^{r \times k}$ with $r \ll \min(d, k)$, just as PCA projects through $\mathbf{V}_k$ of rank $k \ll d$.